In [ ]:
%run -i code/utils.py

# 1. Reading Elevation File

In [ ]:
name_of_file = 'Data/chimmey_tops_dem_1m_utm17n.csv'
Mst_original = read_elevation(name_of_file)

# 2. Creating the FDS file
## 2.1 Defining the Region Where the Simulation Takes Place

In [ ]:
Min_x = 400
Max_x = 600
Min_y = 0
Max_y = 200
Mst   = Mst_original.copy()
Mst = Mst[(Mst.x > Min_x-1) & (Mst.x < Max_x) & (Mst.y > Min_y-1) & (Mst.y < Max_y)]
Mst_o = Mst.copy()


# 2.2 Defining the Resolution of the Meshes and the obstacles in Meters

In [ ]:
R = 4
Mst = Mst[(Mst.x%R==0) & (Mst.y%R==0)]
Ro = 4
Mst_o = Mst_o[(Mst_o.x%Ro==0) & (Mst_o.y%Ro==0)]
Mst = Mst.reset_index(drop=True)
Mst_o = Mst_o.reset_index(drop=True)

## 2.3 Calculating the Range of the Elevation 

In [ ]:
# Minimum and Maximum elevation
Min_z = math.floor(Mst['Elevation'].min())
Max_z = math.ceil(Mst['Elevation'].max())


# 2.4 Writing the FDS File 

In [ ]:
# Defining the time variables
T_begin  = 0.0
T_end    = 60.0
DT       = 0.1
Nmx      = 2    # Number of meshes in x-direction 
Nmy      = 2     # Number pf meshes in y-direction
Nmz      = 2     # Nmber of Meshes  in z-direction
PC       = 1
Location = [500,100] # Location of the fire
Child    = 'Region1'
HRRPUA   = 1500

Indice = Mst_o[(Mst_o['x'] < Location[0]+5) & (Mst_o['x'] > Location[0]-5) & (Mst_o['y'] < Location[1]+5) & (Mst_o['y'] > Location[1]-5)].index
Indice2 = [i for i in Mst_o.index if i not in Indice.values]

foldername = f"Chimney"
filename = f"{Child}.fds"

write_fds_file(T_begin, T_end, DT, PC, Nmx, Nmy, Nmz, Indice, Indice2, Child, HRRPUA)

# 3 Running the FDS program

## 3.1 Defining the folder where the fds input file is

In [ ]:
PATH =  os.getcwd()
Bin_folder = f"{PATH}/Bin"
FDS_FOLDER = f"FDSFiles/{foldername}"
os.chdir(FDS_FOLDER)

## 3.2 Running the fds command

In [ ]:
number_of_process = 4

In [ ]:
bash([f'{PATH}/code/omp_threads.sh'])

In [ ]:
bash(['mpiexec', '-n', number_of_process,'fds',filename])

# 4 Converting the HRR Information from the plot3d Files to CSV Files

In [ ]:
os.chdir(PATH)
Number_of_meshes = Nmx*Nmy*Nmz

reading_hrr(Child, Number_of_meshes)

# 5 Extracting the HRR from the CSV Files

## 5.1 Converting the CSV Files into Dataframes 

In [ ]:
os.chdir(PATH)
Hrr = pd.DataFrame()
for i in range(1,Number_of_meshes+1):
    temp = pd.read_csv(f"{FDS_FOLDER}/{Child}_{i}.csv")
    temp = temp.drop(temp.index[0])
    temp = temp.reset_index(drop=True)
    Hrr  = Hrr.append(temp)
    Hrr = Hrr.reset_index(drop=True)
Hrr = Hrr[[Hrr.columns[0],Hrr.columns[1],Hrr.columns[2],Hrr.columns[7]]]
Hrr.columns = ['x','y','z','hrr']

## 5.2 Cleaning the Data and Matching the Format of Numbers

In [ ]:
Hrr["hrr"] = pd.to_numeric(Hrr["hrr"], downcast="float")
Hrr["x"] = pd.to_numeric(Hrr["x"], downcast="float")
Hrr["y"] = pd.to_numeric(Hrr["y"], downcast="float")

Mst_o["x"] = pd.to_numeric(Mst_o["x"], downcast="float")
Mst_o["y"] = pd.to_numeric(Mst_o["y"], downcast="float")

## 5.3 Extracting the values of HRR greater than 100

In [ ]:
Hrr = Hrr[Hrr['hrr']>100]
Hrr = Hrr.reset_index(drop=True)
HRRPUA   = int(Hrr['hrr'].mean())

## Defining the cells with fire

In [ ]:
Min_x = 300
Max_x = 500
Min_y = 100
Max_y = 300
Mst   = Mst_original.copy()

Mst = Mst[(Mst.x > Min_x-1) & (Mst.x < Max_x) & (Mst.y > Min_y-1) & (Mst.y < Max_y)]
Mst_o = Mst.copy()

Mst = Mst[(Mst.x%R==0) & (Mst.y%R==0)]
Mst_o = Mst_o[(Mst_o.x%Ro==0) & (Mst_o.y%Ro==0)]
Mst = Mst.reset_index(drop=True)
Mst_o = Mst_o.reset_index(drop=True)

# Minimum and Maximum elevation
Min_z = math.floor(Mst['Elevation'].min())
Max_z = math.ceil(Mst['Elevation'].max())

In [ ]:
Temp_indice = [i for i in Mst_o.index if (Mst_o['x'].iloc[i] in Hrr['x'].values) & (Mst_o['y'].iloc[i] in Hrr['y'].values)]

In [ ]:
Indice = Mst_o.iloc[Temp_indice].index
Indice2 = [i for i in Mst_o.index if i not in Indice.values]

In [ ]:
Indice.shape

In [ ]:
Child = 'Region2'
T_begin = T_end
T_end   = T_begin + 60
filename = f"{Child}.fds"


In [ ]:
os.chdir(PATH)
write_fds_file(T_begin, T_end, DT, PC, Nmx, Nmy, Nmz, Indice, Indice2, Child, HRRPUA)
os.chdir(FDS_FOLDER)
bash(['mpiexec', '-n', number_of_process,'fds',filename])